In [ ]:
import os
import time
import gc
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Fuerza CPU
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from numpy import asarray

import mtcnn
from mtcnn import MTCNN

import tensorflow as tf
import keras_vggface

print(f"mtcnn version: {mtcnn.__version__}")
print(f"Tensorflow version: {tf.__version__}")
print(f"Keras VGGFace version: {keras_vggface.__version__}")

# Silenciar logs de tensorflow
import sys
import contextlib

@contextlib.contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout

# Rutas base
input_base_dir = "data/train/val"
output_base_dir = "data/val_processed"
os.makedirs(output_base_dir, exist_ok=True)

# Detector facial
detector = MTCNN()

def extract_face(image_path, required_size=(224, 224)):
    try:
        pixels = plt.imread(image_path)
        with suppress_stdout():  # 👈 silencia salida temporalmente
            results = detector.detect_faces(pixels)
        #results = detector.detect_faces(pixels)
        if not results:
            return None
        x1, y1, width, height = results[0]['box']
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = x1 + width, y1 + height
        face = pixels[y1:y2, x1:x2]
        image = Image.fromarray(face)
        image = image.resize(required_size)
        face_array = asarray(image)
        return face_array
    except Exception as e:
        print(f"[ERROR] {image_path}: {e}")
        return None

# Procesamiento completo
print("🚀 Iniciando procesamiento de imágenes...")

person_dirs = sorted(os.listdir(input_base_dir))[2:20]
total_persons = len(person_dirs)

# Carpeta base para logs
log_base_dir = "data/logs/val"
os.makedirs(log_base_dir, exist_ok=True)

for person_idx, person_id in enumerate(person_dirs, start=1):
    # Skip si ya está completado
    if os.path.exists(os.path.join(log_base_dir, f"{person_id}_done.txt")):
        print(f"✅ {person_id} ya procesado. Saltando.")
        continue

    person_input_dir = os.path.join(input_base_dir, person_id)
    person_output_dir = os.path.join(output_base_dir, person_id)
    os.makedirs(person_output_dir, exist_ok=True)

    img_files = sorted([
        f for f in os.listdir(person_input_dir)
        if os.path.isfile(os.path.join(person_input_dir, f))
    ])
    total_imgs = len(img_files)
    errores_persona = []

    for idx, img_file in enumerate(img_files, start=1):
        input_path = os.path.join(person_input_dir, img_file)
        output_path = os.path.join(person_output_dir, img_file)

        print(
            f"👤 Procesando entidad {person_idx} / {total_persons}: {person_id}  "
            f"📷 Imagen {idx}/{total_imgs}: {img_file}",
            end='\r'
        )

        face = extract_face(input_path)
        if face is not None:
            Image.fromarray(face).save(output_path)
        else:
            errores_persona.append(input_path)

        # Liberar memoria
        del face, input_path, output_path
        gc.collect()
        #time.sleep(1.0)

    # Guardar errores
    if errores_persona:
        log_path = os.path.join(log_base_dir, f"{person_id}.txt")
        with open(log_path, "w", encoding="utf-8") as f:
            for err in errores_persona:
                f.write(err + "\n")

    # Marcar como completado
    with open(os.path.join(log_base_dir, f"{person_id}_done.txt"), "w") as f:
        f.write("done")

print("🚀Procesamiento de imágenes finalizado.")

2025-05-15 17:18:18.682479: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-15 17:18:23.358371: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/grupo3/miniforge3/envs/facegate_preprocess/lib:
2025-05-15 17:18:23.358439: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-05-15 17:18:34.204253: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinf

mtcnn version: 0.1.0
Tensorflow version: 2.11.0
Keras VGGFace version: 0.6


2025-05-15 17:18:42.364297: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/grupo3/miniforge3/envs/facegate_preprocess/lib:
2025-05-15 17:18:42.364356: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2025-05-15 17:18:42.364393: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (grupo3): /proc/driver/nvidia/version does not exist
2025-05-15 17:18:42.413538: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


🚀 Iniciando procesamiento de imágenes...
👤 Procesando entidad 3 / 18: n000078  📷 Imagen 94/458: 0101_01.jpgg

In [1]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from numpy import asarray

import mtcnn
from mtcnn import MTCNN

import tensorflow as tf
import keras_vggface

# Rutas base
input_base_dir = "data/train/val"

# Esto es para buscar los id de las entidades a ejecutar en cierto rango
person_dirs = sorted(os.listdir(input_base_dir))[:20]
for person_idx, person_id in enumerate(person_dirs, start=1):
    print(person_idx, person_id)

2025-05-15 16:51:06.093980: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-15 16:51:06.214618: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/grupo3/miniforge3/envs/facegate_preprocess/lib:
2025-05-15 16:51:06.214645: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-05-15 16:51:07.032270: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinf

1 n000001
2 n000009
3 n000029
4 n000040
5 n000078
6 n000082
7 n000106
8 n000129
9 n000148
10 n000149
11 n000178
12 n000259
13 n000284
14 n000363
15 n000394
16 n000410
17 n000452
18 n000480
19 n000527
20 n000596
21 n000624


In [ ]:
me salte la n009